# Machine Translation Project
## Introduction
In this notebook, we will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Our completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - We'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After investigating the performance of the basic types of neural networks that are often used for machine translation, we will put together a final model which will outperform all other models!
- **Prediction** Run the model on English text.

In [2]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [3]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, LSTM, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


## Dataset
We begin by investigating the dataset that will be used to train and evaluate our pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary. We'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file.

In [6]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  Let's view the first two lines from each file.

In [7]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save us some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [8]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, we won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

We will use Keras's [`Tokenizer`] function to turn each sentence into a sequence of words ids.

In [9]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    
    return x_tk.texts_to_sequences(x), x_tk
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Let's make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [10]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(seq) for seq in x])
    return pad_sequences(x, maxlen=length, padding="post")

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [11]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, we will experiment with various neural network architectures.
We will begin by training four architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN (with attention)

After experimenting with the four architectures, we will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.

In [12]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN 
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, we'll build a RNN that translates English to French.

In [12]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    input_seq = Input(shape=input_shape[1:])
    rnn = GRU(128, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 80us/step - loss: 3.0054 - acc: 0.4366 - val_loss: 2.3240 - val_acc: 0.4821
Epoch 2/10
110288/110288 [==============================] - 8s 76us/step - loss: 2.0565 - acc: 0.5273 - val_loss: 1.8160 - val_acc: 0.5654
Epoch 3/10
110288/110288 [==============================] - 8s 76us/step - loss: 1.6979 - acc: 0.5801 - val_loss: 1.6060 - val_acc: 0.5894
Epoch 4/10
110288/110288 [==============================] - 8s 76us/step - loss: 1.5458 - acc: 0.6014 - val_loss: 1.4902 - val_acc: 0.6098
Epoch 5/10
110288/110288 [==============================] - 8s 76us/step - loss: 1.4463 - acc: 0.6161 - val_loss: 1.4041 - val_acc: 0.6196
Epoch 6/10
110288/110288 [==============================] - 8s 75us/step - loss: 1.3678 - acc: 0.6263 - val_loss: 1.3322 - val_acc: 0.6364
Epoch 7/10
110288/110288 [==============================] - 8s 76us/step - loss: 1.3019 - acc: 0.6386 - val_loss: 1

### Model 2: Embedding
![RNN](images/embedding.png)
We've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, we'll create a RNN model using embedding.

In [15]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    input_seq = Input(shape=input_shape[1:])
    emb = Embedding(english_vocab_size, 200, input_length=output_sequence_length)(input_seq)
    rnn = GRU(128, return_sequences=True)(emb)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)


# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(embed_rnn_model.predict(tmp_x[0]))
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 98us/step - loss: 3.3246 - acc: 0.4235 - val_loss: 2.4486 - val_acc: 0.4878
Epoch 2/10
110288/110288 [==============================] - 10s 92us/step - loss: 1.8969 - acc: 0.5821 - val_loss: 1.3749 - val_acc: 0.6788
Epoch 3/10
110288/110288 [==============================] - 10s 92us/step - loss: 1.1175 - acc: 0.7343 - val_loss: 0.9188 - val_acc: 0.7701
Epoch 4/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.8015 - acc: 0.7906 - val_loss: 0.6995 - val_acc: 0.8091
Epoch 5/10
110288/110288 [==============================] - 10s 91us/step - loss: 0.6354 - acc: 0.8223 - val_loss: 0.5733 - val_acc: 0.8367
Epoch 6/10
110288/110288 [==============================] - 10s 90us/step - loss: 0.5340 - acc: 0.8456 - val_loss: 0.4955 - val_acc: 0.8558
Epoch 7/10
110288/110288 [==============================] - 10s 90us/step - loss: 0.4645 - acc: 0.8636 - val_

### Model 3: Bidirectional RNNs
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [14]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    input_seq = Input(shape=input_shape[1:])
    bd_rnn = Bidirectional(GRU(128, return_sequences=True))(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(bd_rnn)
    
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train and Print prediction(s)
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 13s 116us/step - loss: 2.5655 - acc: 0.5112 - val_loss: 1.7451 - val_acc: 0.5778
Epoch 2/10
110288/110288 [==============================] - 12s 111us/step - loss: 1.5640 - acc: 0.5986 - val_loss: 1.4368 - val_acc: 0.6139
Epoch 3/10
110288/110288 [==============================] - 12s 110us/step - loss: 1.3683 - acc: 0.6238 - val_loss: 1.3089 - val_acc: 0.6369
Epoch 4/10
110288/110288 [==============================] - 12s 112us/step - loss: 1.2681 - acc: 0.6446 - val_loss: 1.2296 - val_acc: 0.6547
Epoch 5/10
110288/110288 [==============================] - 12s 112us/step - loss: 1.1992 - acc: 0.6589 - val_loss: 1.1690 - val_acc: 0.6637
Epoch 6/10
110288/110288 [==============================] - 12s 111us/step - loss: 1.1437 - acc: 0.6677 - val_loss: 1.1178 - val_acc: 0.6728
Epoch 7/10
110288/110288 [==============================] - 12s 110us/step - loss: 1.0948 - acc: 0.6776

### Model 4: Encoder-Decoder
Time to look at encoder-decoder models.

Create an encoder-decoder model in the cell below.

### Attention Mechanism and Encoder-Decoder Model Architecture

The Attenention mechanism implemented below is similar to what is described in the paper "Neural Machine Translation by Jointly Learning to Align and Translate" written by Dzmitry Bahdanau. The Keras implementation of the encoder-decoder model shown below is inspired by the model in the project "Neural Machine Tranlation with Attention" from the course "Sequence Model" offered by deeplearning.ai. However, the model in the "Neural Machine Translation with Attention" project is designed for a Date/Time format converter application. We modified part of that model (adding a word embedding layer, etc) to better fit our language translation application. 

The diagram below shows what one "Attention" step does to calculate the attention variables $\alpha^{\langle t, t' \rangle}$, which are used to compute the context variable $context^{\langle t \rangle}$ for each timestep in the output ($t=1, \ldots, T_y$). Note: the number of time steps in encoder network = $T_x$, and the number of time steps in decoder network = $T_y$.

<img src="images/attn_mechanism.png" style="width:450;height:300px;">

<caption><center> Neural machine translation with attention</center></caption>

<caption><center> Source of the figure: "Sequence Model", deeplearning.ai.<caption><center>

Some notes about the architecture of our encoder-decoder model: 

- The first layer of the model is a word embedding layer which works well with language/text related tasks like our English/French translation application.

- There are two separate LSTMs in this model. The first one is a Bi-directional LSTM which comes *before* the attention mechanism. It's for the encoder. The second LSTM is a uni-directional LSTM which comes *after* the attention mechanism. It's for the decoder. The encoder LSTM goes through $T_x$ time steps; the decoder LSTM goes through $T_y$ time steps. 

- The decoder LSTM passes $s^{\langle t \rangle}, c^{\langle t \rangle}$ from one time step to the next.

- We use $a^{\langle t \rangle} = [\overrightarrow{a}^{\langle t \rangle}; \overleftarrow{a}^{\langle t \rangle}]$ to represent the concatenation of the activations of both the forward-direction and backward-directions of the encoder Bi-LSTM. 

- The above diagram uses a `RepeatVector` node to copy $s^{\langle t-1 \rangle}$'s value $T_x$ times, and then `Concatenation` to concatenate $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ to compute a intermediate variable, which is then passed through a softmax to compute $\alpha^{\langle t, t' \rangle}$ (attention weights).

There are two major functions in the encoder-decoder system: `one_step_attention()` and `model()`.

**1) `one_step_attention()`**: At step $t$, given all the hidden states of the Bi-LSTM ($[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$) and the previous hidden state of the second LSTM ($s^{<t-1>}$), `one_step_attention()` will compute the attention weights ($[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$) and output the context vector (see above figure for details):
$$context^{<t>} = \sum_{t' = 0}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$ 
  
**2) `model()`**: Implements the entire model. It starts by feeding the training data into a word embedding layer, then runs the output from the word embedding through a Bi-LSTM to get back $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$. Then, it calls `one_step_attention()` $T_y$ times (`for` loop). At each iteration of this loop, it gives the computed context vector $c^{<t>}$ to the second LSTM, and runs the output of the LSTM through a dense layer with softmax activation to generate a prediction $\hat{y}^{<t>}$. 

**Note:** The function `model()` will call the layers in `one_step_attention()` $T_y$ using a for-loop, and it is important that all $T_y$ copies have the same weights. I.e., it should not re-initiaiize the weights every time. In other words, all $T_y$ steps should have shared weights. Here's how we implement layers with shareable weights in Keras:
1. Define the layer objects (as global variables).
2. Call these objects when propagating the input.

In [3]:
from keras.layers import RepeatVector, Concatenate, Dot

Using TensorFlow backend.


In [12]:
# Defined shared layers as global variables
repeator = RepeatVector(max_english_sequence_length)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)

In [13]:
# One_step_attention

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a) - m is the number of examples, 
         Tx = time steps used in encoder network, n_a is the size of hidden units in encoder network. 
    
    s_prev -- previous hidden state of the decoder LSTM, numpy-array of shape (m, n_s) - m is the number of examples,
              n_s is the size of hidden units used in decoder network.
    
    Returns:
    context -- context vector, input of the next decoder LSTM cell
    """

    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that we can concatenate it with all hidden states "a".
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis.
    concat = concatenator([a, s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e.
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies.
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas".
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next decoder LSTM-cell.
    context = dotor([alphas, a])
    
    return context

In [14]:
n_enc_hidden_units = 128 # Bi-directional LSTM for encoder network.
n_dec_hidden_units = 256 # Uni-directional LSTM for decoder network.
dec_lstm_cell = LSTM(n_dec_hidden_units, return_state = True)
output_layer = Dense(french_vocab_size+1, activation='softmax')

In [15]:
def encdec_model(input_shape, max_enc_output_len, max_dec_output_len, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param max_enc_output_len: Length of the output sequence in the encoder network
    :param max_dec_output_len: Length of the output sequence in the decoder network
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    input_seq = Input(shape=input_shape[1:])
    s0 = Input(shape=(n_dec_hidden_units,), name='s0')
    c0 = Input(shape=(n_dec_hidden_units,), name='c0')
    s = s0  # Initial hidden state in decoder
    c = c0  # Initial cell state in decoder
    
    outputs = []
    
    emb = Embedding(english_vocab_size, 300, input_length=max_enc_output_len)(input_seq)
    enc_lstm = Bidirectional(LSTM(n_enc_hidden_units, return_sequences=True))(emb)
    
    for t in range(max_dec_output_len):
        # Perform one step of the attention mechanism to get back the context vector at step t.
        context = one_step_attention(enc_lstm, s)
        
        # Apply the decoder LSTM cell to the "context" vector.
        # Pass initial_state = [hidden state, cell state].
        s, _, c = dec_lstm_cell(inputs=context, initial_state = [s, c])
        
        # Apply Dense layer to the hidden state output of the decoder LSTM.
        out = output_layer(s)
        
        # Append "out" to the "outputs" list
        outputs.append(out)

    model = Model(inputs=[input_seq, s0, c0], outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
#tests.test_encdec_model(encdec_model2)

# Train the final model
encdec_rnn_model = encdec_model(
    preproc_english_sentences.shape,
    max_english_sequence_length,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

The last step is to define all the inputs and outputs to fit the model:
- We already have X of shape $(m, T_x)$ containing the training examples, where m is the number of examples, $T_x$ is the time steps used in encoder network.
- We need to create `s0` and `c0` to initialize the decoder LSTM with 0s.
- Given the `model()` we coded, we need the "outputs" to be a list of $T_y$ elements of shape (m, french_vocab_size+1).

In [15]:
s0 = np.zeros((preproc_english_sentences.shape[0], n_dec_hidden_units))
c0 = np.zeros((preproc_english_sentences.shape[0], n_dec_hidden_units))
outputs = list(preproc_french_sentences.swapaxes(0,1)) # swap number of examples with time steps to meet our output from the model

In [17]:
encdec_rnn_model.fit([preproc_english_sentences, s0, c0], outputs, batch_size=1024, epochs=20, validation_split=0.2)


Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 58s 529us/step - loss: 55.0602 - dense_3_loss_1: 4.4653 - dense_3_loss_2: 3.9536 - dense_3_loss_3: 3.3709 - dense_3_loss_4: 4.1835 - dense_3_loss_5: 4.1753 - dense_3_loss_6: 4.2920 - dense_3_loss_7: 4.0742 - dense_3_loss_8: 3.6142 - dense_3_loss_9: 3.4437 - dense_3_loss_10: 3.0377 - dense_3_loss_11: 3.1095 - dense_3_loss_12: 3.1036 - dense_3_loss_13: 2.8320 - dense_3_loss_14: 2.4685 - dense_3_loss_15: 1.5817 - dense_3_loss_16: 0.9141 - dense_3_loss_17: 0.6236 - dense_3_loss_18: 0.4976 - dense_3_loss_19: 0.4535 - dense_3_loss_20: 0.4373 - dense_3_loss_21: 0.4284 - dense_3_acc_1: 0.0135 - dense_3_acc_2: 0.2013 - dense_3_acc_3: 0.3322 - dense_3_acc_4: 0.1063 - dense_3_acc_5: 0.0707 - dense_3_acc_6: 0.0456 - dense_3_acc_7: 0.0614 - dense_3_acc_8: 0.1526 - dense_3_acc_9: 0.1825 - dense_3_acc_10: 0.2810 - dense_3_acc_11: 0.3032 - dense_3_acc_12: 0.3341 - dense_3_acc_13: 0.4119 - den

110288/110288 [==============================] - 51s 461us/step - loss: 12.0765 - dense_3_loss_1: 0.4465 - dense_3_loss_2: 0.4792 - dense_3_loss_3: 0.5382 - dense_3_loss_4: 1.1415 - dense_3_loss_5: 1.0710 - dense_3_loss_6: 1.0406 - dense_3_loss_7: 0.8980 - dense_3_loss_8: 0.8294 - dense_3_loss_9: 0.8331 - dense_3_loss_10: 0.8212 - dense_3_loss_11: 0.8571 - dense_3_loss_12: 0.8838 - dense_3_loss_13: 0.7975 - dense_3_loss_14: 0.6783 - dense_3_loss_15: 0.4446 - dense_3_loss_16: 0.2046 - dense_3_loss_17: 0.0792 - dense_3_loss_18: 0.0232 - dense_3_loss_19: 0.0057 - dense_3_loss_20: 0.0023 - dense_3_loss_21: 0.0015 - dense_3_acc_1: 0.8700 - dense_3_acc_2: 0.8765 - dense_3_acc_3: 0.8576 - dense_3_acc_4: 0.6589 - dense_3_acc_5: 0.6992 - dense_3_acc_6: 0.6891 - dense_3_acc_7: 0.7688 - dense_3_acc_8: 0.7723 - dense_3_acc_9: 0.7710 - dense_3_acc_10: 0.7781 - dense_3_acc_11: 0.7683 - dense_3_acc_12: 0.7572 - dense_3_acc_13: 0.7765 - dense_3_acc_14: 0.7928 - dense_3_acc_15: 0.8617 - dense_3_acc_16:

110288/110288 [==============================] - 51s 461us/step - loss: 4.6861 - dense_3_loss_1: 0.1413 - dense_3_loss_2: 0.1586 - dense_3_loss_3: 0.1828 - dense_3_loss_4: 0.3391 - dense_3_loss_5: 0.3736 - dense_3_loss_6: 0.4012 - dense_3_loss_7: 0.3553 - dense_3_loss_8: 0.3418 - dense_3_loss_9: 0.3413 - dense_3_loss_10: 0.3424 - dense_3_loss_11: 0.3478 - dense_3_loss_12: 0.3563 - dense_3_loss_13: 0.3408 - dense_3_loss_14: 0.2890 - dense_3_loss_15: 0.2050 - dense_3_loss_16: 0.1065 - dense_3_loss_17: 0.0450 - dense_3_loss_18: 0.0139 - dense_3_loss_19: 0.0031 - dense_3_loss_20: 8.5591e-04 - dense_3_loss_21: 4.0734e-04 - dense_3_acc_1: 0.9707 - dense_3_acc_2: 0.9656 - dense_3_acc_3: 0.9557 - dense_3_acc_4: 0.9087 - dense_3_acc_5: 0.8976 - dense_3_acc_6: 0.8880 - dense_3_acc_7: 0.9065 - dense_3_acc_8: 0.9064 - dense_3_acc_9: 0.9066 - dense_3_acc_10: 0.9114 - dense_3_acc_11: 0.9113 - dense_3_acc_12: 0.9090 - dense_3_acc_13: 0.9088 - dense_3_acc_14: 0.9219 - dense_3_acc_15: 0.9426 - dense_3_

110288/110288 [==============================] - 51s 462us/step - loss: 2.7674 - dense_3_loss_1: 0.0818 - dense_3_loss_2: 0.0916 - dense_3_loss_3: 0.1135 - dense_3_loss_4: 0.2068 - dense_3_loss_5: 0.2344 - dense_3_loss_6: 0.2392 - dense_3_loss_7: 0.2165 - dense_3_loss_8: 0.2056 - dense_3_loss_9: 0.1962 - dense_3_loss_10: 0.1901 - dense_3_loss_11: 0.1977 - dense_3_loss_12: 0.2053 - dense_3_loss_13: 0.1970 - dense_3_loss_14: 0.1656 - dense_3_loss_15: 0.1211 - dense_3_loss_16: 0.0647 - dense_3_loss_17: 0.0281 - dense_3_loss_18: 0.0092 - dense_3_loss_19: 0.0023 - dense_3_loss_20: 4.7878e-04 - dense_3_loss_21: 2.0587e-04 - dense_3_acc_1: 0.9802 - dense_3_acc_2: 0.9776 - dense_3_acc_3: 0.9706 - dense_3_acc_4: 0.9469 - dense_3_acc_5: 0.9362 - dense_3_acc_6: 0.9350 - dense_3_acc_7: 0.9387 - dense_3_acc_8: 0.9417 - dense_3_acc_9: 0.9456 - dense_3_acc_10: 0.9506 - dense_3_acc_11: 0.9508 - dense_3_acc_12: 0.9471 - dense_3_acc_13: 0.9477 - dense_3_acc_14: 0.9549 - dense_3_acc_15: 0.9655 - dense_3_

In [23]:
def dec_preds_to_text(predictions, tokenizer):
    """
    Turn predictions (for one example) from an encoder-decoder model into text using the tokenizer.
    :param logits: A list which has max_french_sequence_length (= number of time steps) elements. The shape of each element
                   is (1, french_vocab_size+1).
    
    :param tokenizer: Keras Tokenizer fit on the label.
    
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    
    results = []
    for i in range(len(predictions)): # loop thru all the time steps.
        for pred in np.argmax(predictions[i], 1): # Has only 1 pred
            results.append(index_to_words[pred])

    return ' '.join(results)

In [24]:
# Print prediction(s)
predictions = encdec_rnn_model.predict([preproc_english_sentences[:1], s0, c0])
print(dec_preds_to_text(predictions, french_tokenizer))

new jersey est parfois calme pendant l' de et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


**Observation:** the result is pretty good!

### Model 5: Custom
Based on our experiment, we put together a final model which will outperform all other models!

In [13]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    input_seq = Input(shape=input_shape[1:])
    emb = Embedding(english_vocab_size, 300, input_length=output_sequence_length)(input_seq)
    bd_rnn = Bidirectional(GRU(256, return_sequences=True))(emb)
    logits = TimeDistributed(Dense(french_vocab_size))(bd_rnn)
    
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')

# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train the final model
final_rnn_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 24s 221us/step - loss: 2.5537 - acc: 0.5095 - val_loss: 1.4741 - val_acc: 0.6442
Epoch 2/20
110288/110288 [==============================] - 24s 217us/step - loss: 1.0785 - acc: 0.7354 - val_loss: 0.8125 - val_acc: 0.7861
Epoch 3/20
110288/110288 [==============================] - 24s 217us/step - loss: 0.6629 - acc: 0.8181 - val_loss: 0.5405 - val_acc: 0.8474
Epoch 4/20
110288/110288 [==============================] - 24s 217us/step - loss: 0.4654 - acc: 0.8640 - val_loss: 0.3990 - val_acc: 0.8800
Epoch 5/20
110288/110288 [==============================] - 24s 217us/step - loss: 0.3550 - acc: 0.8932 - val_loss: 0.3116 - val_acc: 0.9059
Epoch 6/20
110288/110288 [==============================] - 24s 217us/step - loss: 0.2822 - acc: 0.9152 - val_loss: 0.2592 - val_acc: 0.9215
Epoch 7/20
110288/110288 [==============================] - 24s 216us/step - loss: 0

## Prediction

In [14]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    model = final_rnn_model


    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=y.shape[1], padding='post')
    sentences = np.array([sentence[0], tmp_x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Future Enhancements

This project focuses on experiment various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. We may use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. However, our final model should still perform very well - we can tell from the validataion accuracy!